In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from graph_generation import create_ER, dataset_nclass_ER, dataset_regression_ER
from models import GCN
from train import Trainer, Dataset
from embedding import Embedding
from experiments import experiment_embedding, train_take_embedding, plot_dim1, plot_dimN, plot_correlation_error
from config_valid import Config, TrainingMode

import torch
from torch_geometric.loader import DataLoader
from torch.utils.tensorboard import SummaryWriter
device = torch.device('cuda')

In [2]:
config_c = Config("configs.yml")
config_c.load_conf()
config_c.valid_conf()



model = GCN(config_c)
model.to('cuda')
print(model)

modo = config_c.get_mode()
if modo == TrainingMode.mode1 or modo == TrainingMode.mode2:
    dataset, list_p = dataset_nclass_ER(config_c)
if modo == TrainingMode.mode3:
    dataset, list_p = dataset_regression_ER(config_c)
ds = Dataset.from_super_instance(0.9, 90, 'cuda', config_c.conf, dataset)
ds.prepare()

GCN(
  (convs): ModuleList(
    (0): GCNConv(1, 8)
    (1): GCNConv(8, 1)
  )
  (lin): Linear(in_features=1, out_features=1, bias=True)
  (leakys): ModuleList(
    (0): LeakyReLU(negative_slope=0.03)
    (1): LeakyReLU(negative_slope=0.03)
    (2): LeakyReLU(negative_slope=0.03)
  )
  (mean_pool): MeanAggregation()
  (drop): Dropout(p=0.5, inplace=False)
)
Generating dataset...
Mean connectivity for each node: 14.978 p=0.05
Mean connectivity for each node: 89.633 p=0.3


100%|███████████████████████████████████████████████████████████| 60/60 [00:04<00:00, 14.58it/s]

Tempo impiegato: 4.126057863235474
tensor([0.2958], device='cuda:0') 54
tensor([0.3006], device='cuda:0') 6


In [5]:
whole_data = ds.dataset_pyg
all_data_loader = DataLoader(whole_data, batch_size=ds.bs, shuffle=False)
model.eval()
#trainer.model.train()
batch = next(iter(all_data_loader))
writer = SummaryWriter("./runs/grafi_NN")
writer.add_graph(model, (batch.x, batch.edge_index), verbose=False)
writer.close()

In [4]:
%load_ext tensorboard

In [6]:
%tensorboard --logdir=runs

In [30]:
from torchviz import make_dot

whole_data = trainer.dataset.dataset_pyg
all_data_loader = DataLoader(whole_data, batch_size=trainer.dataset.bs, shuffle=False)
trainer.model.eval()
#trainer.model.train()
batch = next(iter(all_data_loader))
out = trainer.model(batch.x, batch.edge_index, batch.batch, embedding=False)

make_dot(out.mean(), params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

# Esempio con tensorboardX

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [ ]:
dummy_input = (torch.zeros(1, 3),)


class LinearInLinear(nn.Module):
    def __init__(self):
        super(LinearInLinear, self).__init__()
        self.l = nn.Linear(3, 5)

    def forward(self, x):
        return self.l(x)

with SummaryWriter(comment='LinearInLinear') as w:
    w.add_graph(LinearInLinear(), dummy_input, True)


class MultipleInput(nn.Module):
    def __init__(self):
        super(MultipleInput, self).__init__()
        self.Linear_1 = nn.Linear(3, 5)


    def forward(self, x, y):
        return self.Linear_1(x+y)

with SummaryWriter(comment='MultipleInput') as w:
    w.add_graph(MultipleInput(), (torch.zeros(1, 3), torch.zeros(1, 3)), True)

class MultipleOutput(nn.Module):
    def __init__(self):
        super(MultipleOutput, self).__init__()
        self.Linear_1 = nn.Linear(3, 5)
        self.Linear_2 = nn.Linear(3, 7)

    def forward(self, x):
        return self.Linear_1(x), self.Linear_2(x)

with SummaryWriter(comment='MultipleOutput') as w:
    w.add_graph(MultipleOutput(), dummy_input, True)


class MultipleOutput_shared(nn.Module):
    def __init__(self):
        super(MultipleOutput_shared, self).__init__()
        self.Linear_1 = nn.Linear(3, 5)

    def forward(self, x):
        return self.Linear_1(x), self.Linear_1(x)

with SummaryWriter(comment='MultipleOutput_shared') as w:
    w.add_graph(MultipleOutput_shared(), dummy_input, True)


class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

    def forward(self, x):
        return x * 2


model = SimpleModel()
dummy_input = (torch.zeros(1, 2, 3),)

with SummaryWriter(comment='constantModel') as w:
    w.add_graph(model, dummy_input, True)


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        # self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = F.relu(out)
        return out


dummy_input = torch.rand(1, 3, 224, 224)

with SummaryWriter(comment='basicblock') as w:
    model = BasicBlock(3, 3)
    w.add_graph(model, (dummy_input, ), verbose=True)




class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(x) + F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x


class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

dummy_input = Variable(torch.rand(13, 1, 28, 28))

model = Net1()
with SummaryWriter(comment='Net1') as w:
    w.add_graph(model, (dummy_input, ))

model = Net2()
with SummaryWriter(comment='Net2') as w:
    w.add_graph(model, (dummy_input, ))

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = Net1()

    def forward_once(self, x):
        output = self.cnn1(x)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

model = SiameseNetwork()
with SummaryWriter(comment='SiameseNetwork') as w:
    w.add_graph(model, (dummy_input, dummy_input))


dummy_input = torch.Tensor(1, 3, 224, 224)

with SummaryWriter(comment='alexnet') as w:
    model = torchvision.models.alexnet()
    w.add_graph(model, (dummy_input, ))

with SummaryWriter(comment='vgg19') as w:
    model = torchvision.models.vgg19()
    w.add_graph(model, (dummy_input, ))

with SummaryWriter(comment='densenet121') as w:
    model = torchvision.models.densenet121()
    w.add_graph(model, (dummy_input, ))

with SummaryWriter(comment='resnet18') as w:
    model = torchvision.models.resnet18()
    w.add_graph(model, (dummy_input, ))



class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(
            n_categories +
            input_size +
            hidden_size,
            hidden_size)
        self.i2o = nn.Linear(
            n_categories +
            input_size +
            hidden_size,
            output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden, input

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


n_letters = 100
n_hidden = 128
n_categories = 10
rnn = RNN(n_letters, n_hidden, n_categories)
cat = torch.Tensor(1, n_categories)
dummy_input = torch.Tensor(1, n_letters)
hidden = torch.Tensor(1, n_hidden)


out, hidden, input = rnn(cat, dummy_input, hidden)
with SummaryWriter(comment='RNN') as w:
    w.add_graph(rnn, (cat, dummy_input, hidden), verbose=False)



lstm = torch.nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)

with SummaryWriter(comment='lstm') as w:
    w.add_graph(lstm, (torch.randn(1, 3).view(1, 1, -1), hidden), verbose=True)

In [ ]:
import pytest
print('expect error here:')
with pytest.raises(Exception) as e_info:
    dummy_input = torch.rand(1, 1, 224, 224)
    with SummaryWriter(comment='basicblock_error') as w:
        w.add_graph(model, (dummy_input, ))  # error

# Verifica differenza layer dropout

In [5]:
import torch
import torch.nn as nn

class Model1(nn.Module):
    # Model 1 using functional dropout
    def __init__(self, p=0.0):
        super().__init__()
        self.p = p

    def forward(self, inputs):
        return nn.functional.dropout(inputs, p=self.p, training=True)

class Model2(nn.Module):
    # Model 2 using dropout module
    def __init__(self, p=0.0):
        super().__init__()
        self.drop_layer = nn.Dropout(p=p)

    def forward(self, inputs):
        return self.drop_layer(inputs)
model1 = Model1(p=0.1) # functional dropout 
model2 = Model2(p=0.1) # dropout module

In [11]:
# creating inputs
inputs = torch.rand(10)
print(inputs)
# forwarding inputs in train mode
print('Normal (train) model:')
print('Model 1', model1(inputs))
print('Model 2', model2(inputs))
print()

# switching to eval mode
model1.eval()
model2.eval()

# forwarding inputs in evaluation mode
print('Evaluation mode:')
print('Model 1', model1(inputs))
print('Model 2', model2(inputs))
# show model summary
print('Print summary:')
print(model1)
print(model2)

tensor([0.1189, 0.8991, 0.4356, 0.2209, 0.2069, 0.1514, 0.9957, 0.7391, 0.0204,
        0.8390])
Normal (train) model:
Model 1 tensor([0.1321, 0.9990, 0.0000, 0.0000, 0.2299, 0.1682, 1.1063, 0.8212, 0.0227,
        0.9322])
Model 2 tensor([0.1321, 0.9990, 0.4840, 0.2454, 0.2299, 0.1682, 1.1063, 0.8212, 0.0227,
        0.9322])

Evaluation mode:
Model 1 tensor([0.1321, 0.9990, 0.4840, 0.2454, 0.2299, 0.1682, 1.1063, 0.0000, 0.0227,
        0.9322])
Model 2 tensor([0.1189, 0.8991, 0.4356, 0.2209, 0.2069, 0.1514, 0.9957, 0.7391, 0.0204,
        0.8390])
Print summary:
Model1()
Model2(
  (drop_layer): Dropout(p=0.1, inplace=False)
)


# Verifica del pool mean

In [18]:
from torch_geometric.nn.aggr.basic import MeanAggregation
from torch_geometric.nn import global_mean_pool
import torch
import torch.nn as nn

In [26]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.ma = MeanAggregation()

    def forward(self, inputs):
        return ma(inputs)

class Model2(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs):
        return global_mean_pool(inputs, None)
    
model1 = Model1() 
model2 = Model2()

In [27]:
inputs = torch.rand(10,10)

In [ ]:
inputs.mean(axis=0)

tensor([0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
        0.2989])

In [30]:
ma(inputs)

tensor([[0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
         0.2989]])

In [29]:
# forwarding inputs in train mode
print('Normal (train) model:')
print('Model 1', model1(inputs))
print('Model 2', model2(inputs))
print()

# switching to eval mode
model1.eval()
model2.eval()

# forwarding inputs in evaluation mode
print('Evaluation mode:')
print('Model 1', model1(inputs))
print('Model 2', model2(inputs))

# show model summary
print('Print summary:')
print(model1)
print(model2)

Normal (train) model:
Model 1 tensor([[0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
         0.2989]])
Model 2 tensor([[0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
         0.2989]])

Evaluation mode:
Model 1 tensor([[0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
         0.2989]])
Model 2 tensor([[0.5269, 0.5517, 0.4942, 0.4480, 0.5506, 0.4963, 0.6618, 0.4219, 0.5909,
         0.2989]])
Print summary:
Model1(
  (ma): MeanAggregation()
)
Model2()


### codice sorgente per i due metodi

In [ ]:
global_mean_pool
if batch is None:
        return x.mean(dim=-2, keepdim=x.dim() == 2)
    size = int(batch.max().item() + 1) if size is None else size
return scatter(x, batch, dim=-2, dim_size=size, reduce='mean')



def forward(self, x, index=None,ptr=None, dim_size=None, dim=-2):
    return self.reduce(x, index, ptr, dim_size, dim, reduce='mean')

class Aggregation
def reduce(self, x, index=None, ptr=None, dim_size=None, dim=-2, reduce='add'):

    if ptr is not None:
        ptr = expand_left(ptr, dim, dims=x.dim())
        return segment_csr(x, ptr, reduce=reduce)

    assert index is not None
    return scatter(x, index, dim=dim, dim_size=dim_size, reduce=reduce)